In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transactions.csv')

In [ ]:
df

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,1102483,2018-07-25 00:00:29,1111,2328,40.77,9936029,115,0,0,...,24.0,29.191250,105.0,30.141810,1.0,0.0,10.0,0.0,32.0,0.0
1,1,1102484,2018-07-25 00:01:08,676,6846,9.62,9936068,115,0,0,...,15.0,19.151333,61.0,18.253607,2.0,0.0,5.0,0.0,27.0,0.0
2,2,1102485,2018-07-25 00:01:35,402,4771,81.55,9936095,115,0,0,...,11.0,183.875455,50.0,112.001800,0.0,0.0,12.0,0.0,37.0,0.0
3,3,1102486,2018-07-25 00:01:43,4218,863,23.10,9936103,115,0,0,...,24.0,21.201667,93.0,19.987849,2.0,0.0,6.0,0.0,22.0,0.0
4,4,1102487,2018-07-25 00:02:26,3711,3599,59.25,9936146,115,0,0,...,27.0,80.427407,110.0,74.158455,1.0,0.0,10.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,201290,1303773,2018-08-14 23:57:03,460,6133,16.72,11750223,135,0,0,...,7.0,19.210000,21.0,22.987619,1.0,0.0,3.0,0.0,20.0,0.0
201291,201291,1303774,2018-08-14 23:58:24,3101,3229,38.16,11750304,135,0,0,...,28.0,46.151429,97.0,45.018144,2.0,0.0,13.0,0.0,42.0,0.0
201292,201292,1303775,2018-08-14 23:58:24,4783,7511,69.85,11750304,135,0,0,...,24.0,95.770417,101.0,89.886040,0.0,0.0,3.0,0.0,34.0,0.0
201293,201293,1303776,2018-08-14 23:58:45,2883,8550,14.99,11750325,135,0,0,...,19.0,18.450000,77.0,15.848052,3.0,0.0,7.0,0.0,20.0,0.0


# 1) Ingeniería de Atributos

###Preprocesamiento

In [ ]:
df_copia = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_copia

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,1102483,2018-07-25 00:00:29,1111,2328,40.77,9936029,115,0,0,0,...,24.0,29.191250,105.0,30.141810,1.0,0.0,10.0,0.0,32.0,0.0
1,1102484,2018-07-25 00:01:08,676,6846,9.62,9936068,115,0,0,0,...,15.0,19.151333,61.0,18.253607,2.0,0.0,5.0,0.0,27.0,0.0
2,1102485,2018-07-25 00:01:35,402,4771,81.55,9936095,115,0,0,0,...,11.0,183.875455,50.0,112.001800,0.0,0.0,12.0,0.0,37.0,0.0
3,1102486,2018-07-25 00:01:43,4218,863,23.10,9936103,115,0,0,0,...,24.0,21.201667,93.0,19.987849,2.0,0.0,6.0,0.0,22.0,0.0
4,1102487,2018-07-25 00:02:26,3711,3599,59.25,9936146,115,0,0,0,...,27.0,80.427407,110.0,74.158455,1.0,0.0,10.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,1303773,2018-08-14 23:57:03,460,6133,16.72,11750223,135,0,0,0,...,7.0,19.210000,21.0,22.987619,1.0,0.0,3.0,0.0,20.0,0.0
201291,1303774,2018-08-14 23:58:24,3101,3229,38.16,11750304,135,0,0,0,...,28.0,46.151429,97.0,45.018144,2.0,0.0,13.0,0.0,42.0,0.0
201292,1303775,2018-08-14 23:58:24,4783,7511,69.85,11750304,135,0,0,0,...,24.0,95.770417,101.0,89.886040,0.0,0.0,3.0,0.0,34.0,0.0
201293,1303776,2018-08-14 23:58:45,2883,8550,14.99,11750325,135,0,0,0,...,19.0,18.450000,77.0,15.848052,3.0,0.0,7.0,0.0,20.0,0.0


Se elimina la columna "Unnamed: 0".

In [ ]:
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201295 entries, 0 to 201294
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   TRANSACTION_ID                       201295 non-null  int64  
 1   TX_DATETIME                          201295 non-null  object 
 2   CUSTOMER_ID                          201295 non-null  int64  
 3   TERMINAL_ID                          201295 non-null  int64  
 4   TX_AMOUNT                            201295 non-null  float64
 5   TX_TIME_SECONDS                      201295 non-null  int64  
 6   TX_TIME_DAYS                         201295 non-null  int64  
 7   TX_FRAUD                             201295 non-null  int64  
 8   TX_FRAUD_SCENARIO                    201295 non-null  int64  
 9   TX_DURING_WEEKEND                    201295 non-null  int64  
 10  TX_DURING_NIGHT                      201295 non-null  int64  
 11  CUSTOMER_ID_N

No tiene valores nulos.

In [ ]:
df_copia['TRANSACTION_ID'].duplicated().sum()

0

No tiene valores duplicados.

El análisis de outliers tiene sentido hacerlo únicamente sobre los datos de los montos de las transacciones.

In [ ]:
Q1 = df_copia['TX_AMOUNT'].quantile(0.25)
Q3 = df_copia['TX_AMOUNT'].quantile(0.75)
IQR = Q3 - Q1
outliers_monto = df_copia[((df_copia['TX_AMOUNT'] < (Q1 - 1.5 * IQR)) | (df_copia['TX_AMOUNT'] > (Q3 + 1.5 * IQR)))].index

In [ ]:
outliers_monto

Int64Index([    42,     86,     93,    115,    292,    399,    429,    506,
               565,    584,
            ...
            200847, 200870, 200882, 201023, 201030, 201113, 201155, 201239,
            201250, 201288],
           dtype='int64', length=3104)

Hay 3104 datos que se consideran outliers, pero no se eliminarán ya que es importante considerarlos dentro del entrenamiento de modelo porque que son posibles casos de fraude.

In [ ]:
y = df_copia.TX_FRAUD
df_copia = df_copia.drop(['TX_FRAUD'], axis = 1)

In [ ]:
y.unique()

array([0, 1])

Se separa del conjunto de datos la variable dependiente, que en este caso es "TX_FRAUD", porque es la variable sobre la cual se trabajará y entrenará el modelo.

Todas las columnas, excepto 'TX_DATATIME', tienen un tipo de dato numérico, por lo cual, no es necesario aplicar ningún tipo de codificación de variables categóricas.

Se procede a la normalización de los datos utilizando StandarScaler.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_copia_std = scaler.fit_transform(df_copia)
df_columns = df_copia.columns
df_copia_std = pd.DataFrame(df_copia_std, columns = df_columns)

Una vez con los datos normalizados se puede proceder al entrenamiento del modelo.

# 2) Entrenamiento del Modelo

El tipo de modelo predictivo que se hará es en base a un aprendizaje supervisado y la técnica que se utilizará es árbol de decisiones.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
x_train1, x_test1, y_train1, y_test1 = train_test_split(df_copia_std, y, test_size = 0.3, random_status = 42)

In [ ]:
x_train1

In [ ]:
x_test1

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(x_train1, x_test1)

In [ ]:
#predict1 = model.predict(x_test1)
#predict1

In [ ]:
#y_test1

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(df_copia_std, y, test_size = 0.2, random_status = 42)
model.fit(x_train2, x_test2)
#predict2 = model.predict(x_test2)
#predict2

# 3) Validación del Modelo

Se usan las distintas métricas que tienen los árboles de decisiones para evaluar y validar el modelo

### Modelo 1

In [ ]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test1, predict1)

In [ ]:
#from sklearn.metrics import accuracy_score
#accuracy_score(y_test1, predict1)

In [ ]:
#from sklearn.metrics import precision_score
#precision_score(y_test1, predict1)

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(y_test1, predict1))

## Conclusiones